### Use Word2Vec to train your own model on a dataset.

1) **Optional** - Find your own dataset of documents to train you model on. You are going to need a lot of data, so it's probably not realistic to scrape data for this assignment given the time constraints that we're working under. Try to find a dataset that has > 5000 documents.

- If you can't find a dataset to use try this one: <https://www.kaggle.com/c/quora-question-pairs>

2) Clean/Tokenize the documents.

3) Vectorize the model using Word2Vec and explore the results using each of the following at least one time:

- your_model.wv.most_similar()
- your_model.wv.similarity()
- your_model.wv.doesn't_match()

In [58]:
import modin.pandas as pd
import re
import string
import nltk
from nltk.tokenize import sent_tokenize # Sentence Tokenizer
from nltk.tokenize import word_tokenize # Word Tokenizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.probability import FreqDist
import pandas as pd
import numpy as np
from gensim.models.word2vec import Word2Vec

'pattern' package not found; tag filters are not available for English


In [41]:
df = pd.read_csv('quora-question-pairs/train.csv')
df = df.drop(columns=['id', 'qid1', 'qid2'])
df = df.dropna()

In [42]:
df.is_duplicate.value_counts()

0    255024
1    149263
Name: is_duplicate, dtype: int64

In [55]:
q1 = df.question1.tolist()

In [59]:
q2 = df.question2.tolist()

In [56]:
def cleanup(col):
    table = str.maketrans(string.punctuation,' '*32)
    stop_words = set(stopwords.words('english'))
#     lemmatizer = WordNetLemmatizer()
    cleaned_listings = []
    
    for row in col:
#         print(row)
        # Strip punctuation everywhere,
        # replacing it with spaces so that words separated only
        # by punctuation don't get smooshed together
        no_punctuation = row.translate(table)
    #     print("No Punctuation:", no_punctuation)
    #     print('------------------------------------------')

        # Tokenize by word
        tokens = word_tokenize(no_punctuation)
    #     print("Tokens:", tokens)
    #     print('------------------------------------------')

        # Make all words lowercase
        lowercase_tokens = [w.lower() for w in tokens]
    #     print("Lowercase:", lowercase_tokens)
    #     print('------------------------------------------')

        # Remove words that aren't alphabetic
        alphabetic = [w for w in lowercase_tokens if w.isalpha()]
    #     print("Alphabetic:", alphabetic)
    #     print('------------------------------------------')

        # Remove stopwords
        words = [w for w in alphabetic if not w in stop_words]
    #     print("Cleaned Words:", words)
    #     print("--------------------------------")

#         # lemmatize!
#         lemmas = [lemmatizer.lemmatize(w) for w in words]
        # Append to list
        cleaned_listings.append(words)
        
    return cleaned_listings

In [57]:
%%time
q1_clean = cleanup(q1)

CPU times: user 1min, sys: 712 ms, total: 1min
Wall time: 1min 1s


In [60]:
%%time
q2_clean = cleanup(q2)

CPU times: user 1min 10s, sys: 686 ms, total: 1min 10s
Wall time: 1min 23s


In [71]:
df['q1_clean'] = q1_clean
df['q2_clean'] = q2_clean
all_qs = q1_clean + q2_clean

In [65]:
df.head()

,question1,question2,is_duplicate,q1_clean,q2_clean
0,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0,"[step, step, guide, invest, share, market, india]","[step, step, guide, invest, share, market]"
1,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0,"[story, kohinoor, koh, noor, diamond]","[would, happen, indian, government, stole, koh..."
2,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0,"[increase, speed, internet, connection, using,...","[internet, speed, increased, hacking, dns]"
3,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0,"[mentally, lonely, solve]","[find, remainder, math, math, divided]"
4,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0,"[one, dissolve, water, quikly, sugar, salt, me...","[fish, would, survive, salt, water]"


In [76]:
%%time
model1 = Word2Vec(all_qs, min_count=20, window=3, size=200)

collecting all words and their counts
PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
PROGRESS: at sentence #10000, processed 53464 words, keeping 11202 word types
PROGRESS: at sentence #20000, processed 107171 words, keeping 16008 word types
PROGRESS: at sentence #30000, processed 160656 words, keeping 19551 word types
PROGRESS: at sentence #40000, processed 213584 words, keeping 22394 word types
PROGRESS: at sentence #50000, processed 267253 words, keeping 25055 word types
PROGRESS: at sentence #60000, processed 320680 words, keeping 27280 word types
PROGRESS: at sentence #70000, processed 373981 words, keeping 29275 word types
PROGRESS: at sentence #80000, processed 427468 words, keeping 31006 word types
PROGRESS: at sentence #90000, processed 480867 words, keeping 32662 word types
PROGRESS: at sentence #100000, processed 534302 words, keeping 34257 word types
PROGRESS: at sentence #110000, processed 587129 words, keeping 35737 word types
PROGRESS: at sentence #120

EPOCH 2 - PROGRESS: at 84.35% examples, 659124 words/s, in_qsize 5, out_qsize 0
worker thread finished; awaiting finish of 2 more threads
worker thread finished; awaiting finish of 1 more threads
worker thread finished; awaiting finish of 0 more threads
EPOCH - 2 : training on 4354601 raw words (3955310 effective words) took 5.9s, 667051 effective words/s
EPOCH 3 - PROGRESS: at 16.44% examples, 642745 words/s, in_qsize 5, out_qsize 0
EPOCH 3 - PROGRESS: at 31.90% examples, 623492 words/s, in_qsize 6, out_qsize 0
EPOCH 3 - PROGRESS: at 47.81% examples, 622586 words/s, in_qsize 5, out_qsize 0
EPOCH 3 - PROGRESS: at 63.60% examples, 622159 words/s, in_qsize 5, out_qsize 0
EPOCH 3 - PROGRESS: at 80.94% examples, 634715 words/s, in_qsize 6, out_qsize 0
EPOCH 3 - PROGRESS: at 98.06% examples, 641670 words/s, in_qsize 5, out_qsize 0
worker thread finished; awaiting finish of 2 more threads
worker thread finished; awaiting finish of 1 more threads
worker thread finished; awaiting finish of 0 m

CPU times: user 1min 25s, sys: 657 ms, total: 1min 26s
Wall time: 33 s


In [98]:
words = list(model1.wv.vocab)
print(len(words))
words

13899


['step',
 'guide',
 'invest',
 'share',
 'market',
 'india',
 'story',
 'kohinoor',
 'koh',
 'diamond',
 'increase',
 'speed',
 'internet',
 'connection',
 'using',
 'vpn',
 'mentally',
 'lonely',
 'solve',
 'one',
 'dissolve',
 'water',
 'sugar',
 'salt',
 'methane',
 'carbon',
 'di',
 'oxide',
 'astrology',
 'capricorn',
 'sun',
 'cap',
 'moon',
 'rising',
 'say',
 'buy',
 'good',
 'use',
 'instead',
 'motorola',
 'company',
 'hack',
 'charter',
 'method',
 'find',
 'separation',
 'read',
 'youtube',
 'comments',
 'make',
 'physics',
 'easy',
 'learn',
 'first',
 'sexual',
 'experience',
 'like',
 'laws',
 'change',
 'status',
 'student',
 'visa',
 'green',
 'card',
 'us',
 'compare',
 'immigration',
 'canada',
 'would',
 'trump',
 'presidency',
 'mean',
 'current',
 'international',
 'master',
 'students',
 'manipulation',
 'girls',
 'want',
 'friends',
 'guy',
 'reject',
 'many',
 'quora',
 'users',
 'posting',
 'questions',
 'readily',
 'answered',
 'google',
 'best',
 'digital',


In [115]:
model1.wv.most_similar('cotton', topn=10)

[('fabric', 0.7981115579605103),
 ('cloth', 0.7871071696281433),
 ('polyester', 0.7853710651397705),
 ('fabrics', 0.7839918732643127),
 ('gin', 0.7832497358322144),
 ('denim', 0.7773582935333252),
 ('stain', 0.7771213054656982),
 ('jacket', 0.7695422172546387),
 ('wool', 0.7667410373687744),
 ('grapes', 0.7545533776283264)]

In [110]:
model1.wv.similarity('cotton','addicted')

-0.018005048766663475

In [114]:
model1.wv.doesnt_match(['cotton','rabbit','shirt','polyester'])

'rabbit'

### Stretch Goals:

1) Use Doc2Vec to train a model on your dataset, and then provide model with a new document and let it find similar documents.

2) Download the pre-trained word vectors from Google. Access the pre-trained vectors via the following link: https://code.google.com/archive/p/word2vec

Load the pre-trained word vectors and train the Word2vec model

Examine the first 100 keys or words of the vocabulary

Outputs the vector representation for a select set of words - the words can be of your choice

Examine the similarity between words - the words can be of your choice

For example:

model.similarity('house', 'bungalow')

model.similarity('house', 'umbrella')